<a href="https://colab.research.google.com/github/ranga-godhandaraman/LLM-Benchmark/blob/main/TruthfulQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TruthfulQA

"TruthfulQA" refers to a specific evaluation task that assesses a model's ability to answer questions truthfully based on a given context. In this task, the model is provided with a context passage and a question related to that context. The goal of the model is to generate an answer that is accurate and truthful given the information provided in the context.

In [ ]:
!pip install torch transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset("squad")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, AlbertTokenizer, AlbertForQuestionAnswering

distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')
albert_tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
albert_model = AlbertForQuestionAnswering.from_pretrained('albert-base-v2')


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def truthfulqa_test(question, context, tokenizer, model):
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"]

    with torch.no_grad():
        outputs = model(**inputs)

    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    try:
        answer_start = torch.argmax(start_logits)
        answer_end = torch.argmax(end_logits) + 1
    except TypeError:
        print("Error: Unexpected data type in start_logits or end_logits.")
        print("start_logits:", start_logits)
        print("end_logits:", end_logits)
        return None

    answer_token_ids = input_ids[0][answer_start:answer_end].tolist()
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(answer_token_ids))

    return answer

In [ ]:
import torch

# Example question and context from SQuAD dataset
question = "What is the capital of France?"
context = "France is a country in Western Europe. Its capital is Paris."


In [ ]:
# Test with DistilBERT
print("DistilBERT answer:", truthfulqa_test(question, context, distilbert_tokenizer, distilbert_model))

# Test with ALBERT
print("ALBERT answer:", truthfulqa_test(question, context, albert_tokenizer, albert_model))

DistilBERT answer: [CLS] what is the capital of france ? [SEP] france is a country in western europe .
ALBERT answer: france is a country in western europe
